In [23]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

In [24]:
def load_images_from_folder(folder_path, target_size=(64, 64)):
    images = []
    labels = []
    label_mapping = {}  # Mapping from person names to numeric labels
    label_counter = 0

    for person_folder in os.listdir(folder_path):
        person_path = os.path.join(folder_path, person_folder)

        if os.path.isdir(person_path):
            label_mapping[person_folder] = label_counter
            label_counter += 1

            for filename in os.listdir(person_path):
                img_path = os.path.join(person_path, filename)
                img = load_img(img_path, target_size=target_size, color_mode='grayscale')
                img = img_to_array(img) / 255.0
                images.append(img)
                labels.append(label_mapping[person_folder])

    return np.array(images), np.array(labels), label_mapping

In [25]:
main_folder_path = r"C:\Users\Arnold\Downloads\Final Data Set"
X, y, label_mapping = load_images_from_folder(main_folder_path)

In [26]:
CNNmodel = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='softmax')  # Assuming 9 individuals
])

In [27]:
CNNmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [28]:
CNNmodel.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
220/220 [==============================] - 12s 50ms/step - loss: 1.7510 - accuracy: 0.3518
Epoch 2/10
220/220 [==============================] - 12s 52ms/step - loss: 1.0311 - accuracy: 0.6438
Epoch 3/10
220/220 [==============================] - 12s 55ms/step - loss: 0.7653 - accuracy: 0.7341
Epoch 4/10
220/220 [==============================] - 11s 50ms/step - loss: 0.6112 - accuracy: 0.7926
Epoch 5/10
220/220 [==============================] - 11s 51ms/step - loss: 0.5241 - accuracy: 0.8202
Epoch 6/10
220/220 [==============================] - 13s 58ms/step - loss: 0.4504 - accuracy: 0.8501
Epoch 7/10
220/220 [==============================] - 13s 57ms/step - loss: 0.4036 - accuracy: 0.8618
Epoch 8/10
220/220 [==============================] - 13s 58ms/step - loss: 0.3489 - accuracy: 0.8793
Epoch 9/10
220/220 [==============================] - 14s 63ms/step - loss: 0.3122 - accuracy: 0.8908
Epoch 10/10
220/220 [==============================] - 13s 58ms/step - loss: 0.272

In [60]:
model_save_path = r"C:\Users\Arnold\Saved_Models\signature_model.h5"
CNNmodel.save(model_save_path)

In [61]:
from tensorflow.keras.models import load_model

# Load the model
model = load_model(model_save_path)

In [68]:
# Function to load test images (using your image loading approach)
def load_test_images(folder_path, target_size=(64, 64)):
    test_images = []
    filenames = []

    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = load_img(img_path, target_size=target_size, color_mode='grayscale')
        img = img_to_array(img) / 255.0
        test_images.append(img)
        filenames.append(filename)

    return np.array(test_images), filenames

In [77]:
# Load test images
test_folder_path = r"C:\Users\Arnold\OneDrive\Desktop\TESTDATA"
X_test, test_filenames = load_test_images(test_folder_path)

In [78]:
predictions = model.predict(X_test)

1/1 [==============================] - 0s 219ms/step


In [79]:
threshold = 0.6  # Set a threshold for minimum confidence

for i, pred in enumerate(predictions):
    if max(pred) < threshold:
        print(f"The signature in {test_filenames[i]} is of a new person.")
    else:
        predicted_label = np.argmax(pred)
        person_name = [name for name, label in label_mapping.items() if label == predicted_label][0]
        print(f"The signature in {test_filenames[i]} belongs to {person_name}.")

The signature in augmented_signature_16.jpg belongs to khadija.
The signature in sign1_333.jpg belongs to Prerna.
The signature in sign1_383.jpg belongs to khadija.
The signature in sign1_731.jpg belongs to Parita.
The signature in sign1_752.jpg belongs to Indu.
The signature in sign1_754.jpg is of a new person.
The signature in sign1_758.jpg belongs to Ajm.
The signature in sign1_778.jpg belongs to Anurag.
The signature in sign1_82.jpg belongs to Mayur.
